In [1]:
import pandas as pd
import numpy as np
from Levenshtein import distance
import re
import requests
import bs4
import lxml
import os

In [2]:
# getting file names of wav files
files_all = os.listdir("music_data")

In [3]:
# function to get "remaster", "remaster year" and "live performance" information
def get_remaster_year_live(string):
    remastered = False
    year = -1
    live = False
    remaster_index = string.lower().find('remaster')
    if remaster_index != -1:
        remastered = True
        before_begin_index = max(remaster_index - 8, 0)
        before_end_index = remaster_index
        after_begin_index = remaster_index + len('remaster')
        after_end_index = min(remaster_index + len('remaster') + 10, len(string))
        matches = re.findall("[0-9][0-9][0-9][0-9]", string[before_begin_index : before_end_index])
        if len(matches) == 1:
            year = matches[0]
        else:
            matches = re.findall("[0-9][0-9][0-9][0-9]", string[after_begin_index : after_end_index])
            if len(matches) == 1:
                year = matches[0]
    live_index = string.lower().find('live')
    if live_index != -1:
        live = True

    return remastered, year, live


In [4]:
# Creating arrays for generating dataframe with columns showing the following information:
# file name, artist, song, remastered song?, remaster year, live performance?, orginal file name


files = []
artists = []
songs = []
remastered_list = []
year_remastered_list = []
live_list = []
original_filenames = []
for f in files_all:
    s = f.split(" ---- ")
    if len(s) == 3:
        artist = str(s[0]).strip()
        song = str(s[1]).strip()
        original_filename = str(s[2])
        remastered, year, live = get_remaster_year_live(original_filename)
        artists.append(artist)
        songs.append(song)
        remastered_list.append(remastered)
        year_remastered_list.append(year)
        live_list.append(live)
        original_filenames.append(original_filename)
        files.append(f)

In [5]:
# creating the data frame

df_files = pd.DataFrame({ 
    "artist" : artists, 
    "song" : songs,
    "remastered" : remastered_list,
    "remastered_year" : year_remastered_list,
    "live_performance" : live_list,
    "original_file_name" : original_filenames, 
    "file" : files 
})
df_files

,artist,song,remastered,remastered_year,live_performance,original_file_name,file
0,backstreet boys,as long as you love me,False,-1,False,0156. backstreet boys - as long as you love me...,backstreet boys ---- as long as you love me --...
1,ZZ Top,Sharp Dressed Man,True,2008,False,151. ZZ Top - Sharp Dressed Man (2008 Remas...,ZZ Top ---- Sharp Dressed Man ---- 151. ZZ T...
2,the prodigy,firestarter,False,-1,False,0180. the prodigy - firestarter [1996].mp3,the prodigy ---- firestarter ---- 0180. the pr...
3,Status Quo,Spinning Wheel Blues,False,-1,False,356. Status Quo - Spinning Wheel Blues.mp3,Status Quo ---- Spinning Wheel Blues ---- 356....
4,Willie Nelson,Blues Eyes Crying In The Rain,False,-1,False,Willie Nelson - Blues Eyes Crying In The Rain.mp3,Willie Nelson ---- Blues Eyes Crying In The Ra...
...,...,...,...,...,...,...,...
1376,madonna,frozen,False,-1,False,0069. madonna - frozen [1998].mp3,madonna ---- frozen ---- 0069. madonna - froze...
1377,Van Halen,Beautiful Girls,True,2015,False,258. Van Halen - Beautiful Girls (2015 Rema...,Van Halen ---- Beautiful Girls ---- 258. Van...
1378,golden earring,another 45 miles,False,-1,True,0090. golden earring - another 45 miles (live)...,golden earring ---- another 45 miles ---- 009...
1379,David Bowie,Rebel Rebel,True,2016,False,351. David Bowie - Rebel Rebel (2016 Remast...,David Bowie ---- Rebel Rebel ---- 351. David...


In [6]:
#df_files.to_csv('files.csv')

In [ ]:
df_files = pd.read_csv("files.csv")

In [7]:
# loading data frame with key and mode information of many songs
df = pd.read_csv("music_genre.csv")
df

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,4-Apr,0.759,Electronic
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50000,58878.0,BEXEY,GO GETTA,59.0,0.03340,0.913,-1.0,0.574,0.00000,C#,0.119,-7.022,Major,0.2980,98.02799999999999,4-Apr,0.330,Hip-Hop
50001,43557.0,Roy Woods,Drama (feat. Drake),72.0,0.15700,0.709,251860.0,0.362,0.00000,B,0.109,-9.814,Major,0.0550,122.04299999999999,4-Apr,0.113,Hip-Hop
50002,39767.0,Berner,Lovin' Me (feat. Smiggz),51.0,0.00597,0.693,189483.0,0.763,0.00000,D,0.143,-5.443,Major,0.1460,131.079,4-Apr,0.395,Hip-Hop
50003,57944.0,The-Dream,Shawty Is Da Shit,65.0,0.08310,0.782,262773.0,0.472,0.00000,G,0.106,-5.016,Minor,0.0441,75.88600000000001,4-Apr,0.354,Hip-Hop


In [8]:
# function to get information about remastered, remastered year, live performance of each song in the music genre data set
def cleaning_track_name(track_name):
    remastered = False
    year = -1
    live_p = False
    track = str(track_name)
    track_splited = track.split(" - ")
    track = track_splited[0]
    track = re.sub("\(.*\)", "", track)
    track = re.sub("\[.*\]", "", track)
    if len(track_splited) == 2:
        remastered, year, live_p = get_remaster_year_live(track_splited[1])
    return track.strip(), remastered, year, live_p

In [9]:
#for row in df[remaster_filter].reset_index().iterrows():
#    name = row[1]['track_name']
#    name_list = cleaning_track_name(name)
#    print(name, "--", name_list[0], "--", name_list[1], "--", name_list[2])

In [10]:
# Creating arrays for adding the following information (as columns) to the music_genre data frame:
# remastered song?, remaster year, live performance?
track_names_cleaning = df.apply(lambda x: cleaning_track_name(x['track_name']), axis=1)
track_names_cleaning = np.stack(track_names_cleaning.to_numpy())
cleaned_track_names = track_names_cleaning[:,0]
remaster = track_names_cleaning[:,1]
year = track_names_cleaning[:,2]
live_p = track_names_cleaning[:,3]

In [11]:
# adding the columns to the data frame
df['cleaned_track_name'] = cleaned_track_names
df['remastered_true_false'] = remaster
df['remastered_year'] = year
df['live_performance'] = live_p

In [12]:
# checking that new columns are OK
remaster_filter = df.apply(lambda r: True if 'remaster' in str(r['track_name']).lower() else False, axis=1)
df[remaster_filter][['track_name','cleaned_track_name','remastered_true_false','remastered_year', 'live_performance']]

,track_name,cleaned_track_name,remastered_true_false,remastered_year,live_performance
44,World (The Price Of Love) - [Radio Edit] [2015...,World,True,2015,False
65,Sooner Than You Think - 2015 Remastered Version,Sooner Than You Think,True,2015,False
133,Spacelab - 2009 Remastered Version,Spacelab,True,2009,False
276,Kingdom of Rain - Remastered,Kingdom of Rain,True,-1,False
300,Endospore - Remastered,Endospore,True,-1,False
...,...,...,...,...,...
48194,I Got A Story To Tell - 2014 Remastered Version,I Got A Story To Tell,True,2014,False
48306,Hypnotize (feat. Pam Long) - 2016 Remaster,Hypnotize,True,2016,False
48788,"Flava in Ya Ear (feat. The Notorious B.I.G., L...",Flava in Ya Ear,True,2016,False
49678,You're Nobody (Til Somebody Kills You) - 2014 ...,You're Nobody,True,2014,False


In [13]:
# creating a data frame with the information about the matches between the .wav song files and the songs in
# the music genre data frame
df_matches = pd.DataFrame({
    'music_genre_index' : [],
    'files_index' : [],
    'artist_file_len' : [],
    'song_file_len' : [],
    'artist_music_genre_len' : [],
    'song_music_genre_len' : [],
    'distance_artist' : [],
    'distance_song' : [],
    'distance_sum' : []
})

In [14]:
df_matches

,music_genre_index,files_index,artist_file_len,song_file_len,artist_music_genre_len,song_music_genre_len,distance_artist,distance_song,distance_sum


In [15]:
# function to search and add matches to the df_matches data frame
def search_and_add_matches(row_music_genre, row_files, df_matches):
    artist_file = str(row_files['artist']).strip()
    song_file = str(row_files['song']).strip()
    artist_data = str(row_music_genre['artist_name']).strip()
    song_data = str(row_music_genre['cleaned_track_name']).strip()
    distance_song = distance(song_data, song_file)
    distance_artist = distance(artist_data, artist_file)
    distance_sum = distance_song + distance_artist
    if distance_song < 8 and distance_artist < 8:
        new_row = [row_music_genre.name, 
                   row_files.name, 
                   len(artist_file), 
                   len(song_file), 
                   len(artist_data), 
                   len(song_data), 
                   distance_artist, 
                   distance_song, 
                   distance_sum]
        df_matches.loc[len(df_matches)] = new_row

In [16]:
# read matches.csv file instead for avoiding running this cell / see below
# uncomment if there is no matches.csv file

#kk = df.apply(
#    lambda row_music_genre: df_files.apply(
#        lambda row_files: search_and_add_matches(row_music_genre, row_files, df_matches), 
#        axis=1), 
#    axis=1) 

In [17]:
#df_matches

In [18]:
#df_matches.to_csv('matches.csv')

In [19]:
df_matches = pd.read_csv('matches.csv')

In [20]:
len(df_matches)

167134

In [21]:
# funtion to create a filter to get only "good" matches from the df_matches data frame
# this filter is considering only matches with coincidences in remaster and live information, less than
# artist_string_lengh/3 difference using Levenshtein distance between artist names, and less than
# song_string_lengh/3 difference using Levenshtein distance between song names.
def matches_filter(matches_row):
    art_len = matches_row['artist_file_len']
    song_len = matches_row['song_file_len']
    dist_art = matches_row['distance_artist']
    dist_song = matches_row['distance_song']
    remaster_music_genre = df.loc[matches_row['music_genre_index']]['remastered_true_false']
    year_music_genre = df.loc[matches_row['music_genre_index']]['remastered_year']
    live_music_genre = df.loc[matches_row['music_genre_index']]['live_performance']
    remaster_files = df_files.loc[matches_row['files_index']]['remastered']
    year_files = df_files.loc[matches_row['files_index']]['remastered_year']
    live_files = df_files.loc[matches_row['files_index']]['live_performance']
    #print(type(remaster_music_genre), type(remaster_files), type(year_music_genre), type(year_files), type(live_music_genre), type(live_files))
    if (dist_art+1e-5) <= (art_len/3) and \
                (dist_song+1e-5) <= (song_len/3) and \
                 str(remaster_music_genre).lower().strip() == str(remaster_files).lower().strip() and \
                 int(year_music_genre) == int(year_files) and \
                 str(live_music_genre).lower().strip() == str(live_files).lower().strip():
        return True
    else:
        return False

In [23]:
# making the filter
mat_filter_array = df_matches.apply(matches_filter, axis=1)

In [ ]:
# applying the filter
df_matches_filtered = df_matches[mat_filter_array]
df_matches_filtered = df_matches_filtered.reset_index(drop=True)

In [ ]:
# variables to create a data frame for a last human check
indexes = []
artists_music_genre = []
songs_music_genre = []
files = []
for row in df_matches_filtered.iterrows():
    indexes.append(row[0])
    artists_music_genre.append(df.loc[row[1]['music_genre_index']]['artist_name'])
    songs_music_genre.append(df.loc[row[1]['music_genre_index']]['track_name'])
    files.append(df_files.loc[row[1]['files_index']]['original_file_name'])

In [ ]:
# making the data frame
df_show_matches = pd.DataFrame({
    'index' : indexes,
    'artist_music_genre' : artists_music_genre,
    'song_music_genre' : songs_music_genre,
    'file' : files
})
df_show_matches

,index,artist_music_genre,song_music_genre,file
0,0,Everything But The Girl,Missing - Todd Terry Club Mix / US Radio Edit,0315. everything but the girl - missing (todd ...
1,1,The Prodigy,Firestarter,0180. the prodigy - firestarter [1996].mp3
2,2,The Prodigy,Firestarter (Alvin Risk Remix),0180. the prodigy - firestarter [1996].mp3
3,3,Daft Punk,Around The World - Radio Edit,0431. daft punk - around the world [1997].mp3
4,4,Fatboy Slim,Praise You,0401. fatboy slim - praise you [1998].mp3
...,...,...,...,...
502,502,Vanilla Ice,Ice Ice Baby,0215. vanilla ice - ice ice baby [1990].mp3
503,503,Ginuwine,Pony,0490. ginuwine - pony [1996].mp3
504,504,DJ Jazzy Jeff & The Fresh Prince,Summertime,0424. dj jazzy jeff & the fresh prince - summe...
505,505,2Pac,Dear Mama,0116. 2pac - dear mama [1995].mp3


In [ ]:
## saving the data frame for a manual check
#df_show_matches.to_csv('show_matches.csv')

In [ ]:
# Matches checked manually one by one (by a human being). Detected no-matches are shown by an e in the 'e_means_NO_MATCH' column
df_show_matches_checked = pd.read_csv('show_matches_checked.csv')
df_show_matches_checked

,music_genre_index,e_means_NO_MATCH,artist_music_genre,song_music_genre,file
0,0,e,Everything But The Girl,Missing - Todd Terry Club Mix / US Radio Edit,0315. everything but the girl - missing (todd ...
1,1,1,The Prodigy,Firestarter,0180. the prodigy - firestarter [1996].mp3
2,2,e,The Prodigy,Firestarter (Alvin Risk Remix),0180. the prodigy - firestarter [1996].mp3
3,3,e,Daft Punk,Around The World - Radio Edit,0431. daft punk - around the world [1997].mp3
4,4,4,Fatboy Slim,Praise You,0401. fatboy slim - praise you [1998].mp3
...,...,...,...,...,...
502,502,502,Vanilla Ice,Ice Ice Baby,0215. vanilla ice - ice ice baby [1990].mp3
503,503,503,Ginuwine,Pony,0490. ginuwine - pony [1996].mp3
504,504,504,DJ Jazzy Jeff & The Fresh Prince,Summertime,0424. dj jazzy jeff & the fresh prince - summe...
505,505,505,2Pac,Dear Mama,0116. 2pac - dear mama [1995].mp3


In [ ]:
# Creating dataset after a human check
manually_checked_matches_filter = df_show_matches_checked.apply(lambda x: True if x['e_means_NO_MATCH'] != 'e' else False, axis=1)
df_matches_filteredANDchecked = df_matches_filtered[manually_checked_matches_filter.values]
df_matches_filteredANDchecked

,Unnamed: 0,music_genre_index,files_index,artist_file_len,song_file_len,artist_music_genre_len,song_music_genre_len,distance_artist,distance_song,distance_sum
1,5972,888,2,11,11,11,11,2,1,3
4,16106,2363,1326,11,10,11,10,2,2,4
5,16582,2482,895,9,8,9,8,1,1,2
6,20951,3203,637,4,15,4,15,1,3,4
7,23201,3527,1053,11,12,11,12,2,3,5
...,...,...,...,...,...,...,...,...,...,...
502,147761,46969,262,11,12,11,12,2,3,5
503,150050,47304,1295,8,4,8,4,1,1,2
504,158388,48547,892,32,10,32,10,7,1,8
505,161698,49119,1300,4,9,4,9,1,2,3


In [ ]:
# variables to create a new data frame with ONLY the necessary information for the project
music_genre_indexes = []
artists = []
songs = []
keys = []
modes = []
keys_modes = []
file_names = []
cheking_duplicates = np.ones(len(df), dtype=bool)
for r in df_matches_filteredANDchecked.iterrows():
    if cheking_duplicates[r[1]['music_genre_index']]:
        cheking_duplicates[r[1]['music_genre_index']] = False
        df_row = df.loc[r[1]['music_genre_index']]
        music_genre_indexes.append(r[1]['music_genre_index'])
        artists.append(df_row['artist_name'])
        songs.append(df_row['track_name'])
        keys.append(df_row['key'])
        modes.append(df_row['mode'])
        keys_modes.append(df_row['key']+' '+df_row['mode'])
        file_names.append(df_files.loc[r[1]['files_index']]['file'])

In [ ]:
# making the data frame
df_nn_data = pd.DataFrame({
    'music_genre_index' : music_genre_indexes,
    'artist' : artists,
    'song' : songs,
    'key' : keys,
    'mode' : modes,
    'key_and_mode' : keys_modes,
    'file_name' : file_names
})
df_nn_data

,music_genre_index,artist,song,key,mode,key_and_mode,file_name
0,888,The Prodigy,Firestarter,B,Major,B Major,the prodigy ---- firestarter ---- 0180. the pr...
1,2363,Fatboy Slim,Praise You,C#,Major,C# Major,fatboy slim ---- praise you ---- 0401. fatboy ...
2,2482,Faithless,Insomnia,G,Major,G Major,faithless ---- insomnia ---- 0035. faithless -...
3,3203,Moby,Feeling So Real,G,Major,G Major,moby ---- feeling so real ---- 0392. moby - fe...
4,3527,The Prodigy,Out Of Space,A,Major,A Major,the prodigy ---- out of space ---- 0379. the p...
...,...,...,...,...,...,...,...
453,46969,Vanilla Ice,Ice Ice Baby,D,Major,D Major,vanilla ice ---- ice ice baby ---- 0215. vanil...
454,47304,Ginuwine,Pony,G#,Minor,G# Minor,ginuwine ---- pony ---- 0490. ginuwine - pony ...
455,48547,DJ Jazzy Jeff & The Fresh Prince,Summertime,G,Major,G Major,dj jazzy jeff & the fresh prince ---- summerti...
456,49119,2Pac,Dear Mama,F#,Major,F# Major,2pac ---- dear mama ---- 0116. 2pac - dear mam...


In [29]:
## saving the data frame
#df_nn_data.to_csv('neuralNet_feeding_data.csv',index=False)

In [30]:
df_nn_data

,music_genre_index,artist,song,key,mode,key_and_mode,file_name
0,888,The Prodigy,Firestarter,B,Major,B Major,the prodigy ---- firestarter ---- 0180. the pr...
1,2363,Fatboy Slim,Praise You,C#,Major,C# Major,fatboy slim ---- praise you ---- 0401. fatboy ...
2,2482,Faithless,Insomnia,G,Major,G Major,faithless ---- insomnia ---- 0035. faithless -...
3,3203,Moby,Feeling So Real,G,Major,G Major,moby ---- feeling so real ---- 0392. moby - fe...
4,3527,The Prodigy,Out Of Space,A,Major,A Major,the prodigy ---- out of space ---- 0379. the p...
...,...,...,...,...,...,...,...
453,46969,Vanilla Ice,Ice Ice Baby,D,Major,D Major,vanilla ice ---- ice ice baby ---- 0215. vanil...
454,47304,Ginuwine,Pony,G#,Minor,G# Minor,ginuwine ---- pony ---- 0490. ginuwine - pony ...
455,48547,DJ Jazzy Jeff & The Fresh Prince,Summertime,G,Major,G Major,dj jazzy jeff & the fresh prince ---- summerti...
456,49119,2Pac,Dear Mama,F#,Major,F# Major,2pac ---- dear mama ---- 0116. 2pac - dear mam...


In [31]:
df_nn_data.groupby('mode').apply(lambda x: len(x))


mode
Major    370
Minor     88
dtype: int64